In [1]:
%load_ext autoreload
%autoreload 2

import os 
import sys
import time

project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

for path in sys.path:
    print(path)

import numpy as np
import pandas as pd
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import cvxpy as cp

from src.setup_problem_modular import Setup
from src.objective_function import specimen_construct_objective_function                                        
from src.objective_function import specimen_construct_objective_function_loss                                        
from src.misc import visualize_result

/home/onoue/ws/lukasiewicz_1/notebooks/pima_indian_diabetes
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages
/home/onoue/ws/lukasiewicz_1


In [15]:
from src.setup_problem_modular import Setup
from src.objective_function import specimen_construct_objective_function

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules.txt']
}

obj_constructor = specimen_construct_objective_function

constraints_flag_dict = {
    "pointwise"  : False,
    "logical"    : False,
    "consistency": True
}
# constraints_flag_dict = None


c1 = 2.5
c2 = 2.5
# c1 = 100
# c2 = 100

model_name = "model_22222222222222"
# model_name = "model_1aisj"

problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor, name=model_name)
objective, constraints = problem_instance.main(constraints_flag_dict=constraints_flag_dict,
                                               c1=c1,
                                               c2=c2)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')

load_data took 0.10862588882446289 seconds!
load_rules took 0.0009512901306152344 seconds!
specimen_construct_objective_function took 3.006808042526245 seconds!
consistency constraints
__call__ took 9.048030853271484 seconds!


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 10 04:44:04 PM: Your problem has 12014 variables, 24464 constraints, and 0 parameters.
(CVXPY) Dec 10 04:44:05 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 10 04:44:05 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 10 04:44:05 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 10 04:44:08 PM: Compiling problem (target solver=ECOS).
(CVXPY) Dec 10 04:44:08 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuf

In [16]:
from src.evaluation import EvaluateModel

test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
save_file_path = os.path.join(project_dir_path, "outputs/pima_indian_diabetes/result_formated_2.json")

evaluate_instance = EvaluateModel(problem_instance, file_path)
evaluate_instance(save_file_path=save_file_path)

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onoue/w

In [18]:
import pandas as pd
import json
import os

# ファイルパスのリスト
file_paths = ['outputs/pima_indian_diabetes/result_formated_1.json', 'outputs/pima_indian_diabetes/result_formated_2.json']

# データを格納するための空のデータフレームを作成
df = pd.DataFrame()

# ファイルを読み込んでデータフレームに追加
for file_path in file_paths:
    file_path = os.path.join(project_dir_path, file_path)
    with open(file_path, 'r') as file:
        data = json.load(file)
        name = data['name']
        del data['name']
        del data['Confusion_matrix']
        del data['Classification_report']
        df[name] = pd.Series(data)

# 結果を表示
print(df)

                 model_1aisj  model_22222222222222
note                     NaN                   NaN
Accuracy            0.671642              0.671642
Precision           0.000000              0.000000
Recall              0.000000              0.000000
F1-score            0.000000              0.000000
Auc                 0.839394              0.839394
VN_pointwise    11792.000000          11792.000000
VN_logical        280.000000            280.000000
VN_consistency      0.000000              0.000000
VN_all          12072.000000          12072.000000
VR_pointwise        1.000000              1.000000
VR_logical          0.152174              0.152174
VR_consistency      0.000000              0.000000
VR_all              0.316884              0.316884


In [11]:
df

,model_1aisj,model_22222222222222
Note,None,None
Accuracy,0.671642,0.671642
Precision,0.0,0.0
Recall,0.0,0.0
F1-score,0.0,0.0
Auc,0.839394,0.839394
Violation_num,"{'Pointwise': 11792.0, 'Logical': 280.0, 'Cons...","{'Pointwise': 11792.0, 'Logical': 280.0, 'Cons..."
Violation_ratio,"{'Pointwise': 1.0, 'Logical': 0.15217391304347...","{'Pointwise': 1.0, 'Logical': 0.15217391304347..."
